In [1]:
import pandas as pd 
import numpy as np
import random
import os
import sys

from datetime import date, datetime
from timeit import default_timer as timer
import time as tm

!pip install wget
import wget

from zipfile import ZipFile

from bs4 import BeautifulSoup as bs
import requests
import re

import json
!pip install psycopg2
import psycopg2
import sqlalchemy

from io import StringIO

In [2]:
url = "http://200.152.38.155/CNPJ/"
res = requests.get(url)

try:
    res.status_code=200
    print("URL acessível")
except:
    print("URL não acessível. Page Status = "+res.status_code)

URL acessível


In [3]:
# Extraindo conteúdo do site 
html_page = pd.read_html(url)
# Convertando o conteúdo obtido em um dataframe pandas
table_files = html_page[0]
# Excluindo colunas indesejáveis
table_files.drop(["Unnamed: 0", "Description"], axis=1, inplace=True)
# Excluindo linhas vazias (NaN)
table_files=table_files = table_files.dropna().reset_index(drop=True)
# Excluindo outras linhas com conteúdo indesejável
table_files=table_files.drop([21,38]).reset_index(drop=True)
table_files.sort_values(by='Name')

,Name,Last modified,Size
0,Cnaes.zip,2023-06-12 22:05,22K
1,Empresas0.zip,2023-06-12 22:08,257M
2,Empresas1.zip,2023-06-12 22:07,74M
3,Empresas2.zip,2023-06-12 22:08,75M
4,Empresas3.zip,2023-06-12 22:08,81M
5,Empresas4.zip,2023-06-12 22:09,86M
6,Empresas5.zip,2023-06-12 22:09,93M
7,Empresas6.zip,2023-06-12 22:10,90M
8,Empresas7.zip,2023-06-12 22:11,95M
9,Empresas8.zip,2023-06-12 22:10,95M


In [4]:
table_files_series = pd.Series(list(table_files["Name"]))

links=[]

for n in range(0,len(table_files_series)):
    url_completa = url+table_files_series.loc[n]
    links.append(url_completa)

In [5]:
link_cnae = links[0:1]
link_simples = links[26:27]
link_tab_aux = links[21:26]
link_empresa = links[1:11]
link_estabelecimento = links[11:21]
link_socios = links[27:37]

In [6]:
link_estabelecimento

['http://200.152.38.155/CNPJ/Estabelecimentos0.zip',
 'http://200.152.38.155/CNPJ/Estabelecimentos1.zip',
 'http://200.152.38.155/CNPJ/Estabelecimentos2.zip',
 'http://200.152.38.155/CNPJ/Estabelecimentos3.zip',
 'http://200.152.38.155/CNPJ/Estabelecimentos4.zip',
 'http://200.152.38.155/CNPJ/Estabelecimentos5.zip',
 'http://200.152.38.155/CNPJ/Estabelecimentos6.zip',
 'http://200.152.38.155/CNPJ/Estabelecimentos7.zip',
 'http://200.152.38.155/CNPJ/Estabelecimentos8.zip',
 'http://200.152.38.155/CNPJ/Estabelecimentos9.zip']

In [2]:
path = 'C:/Users/thiago.silva/TCC/RFB_CNPJ'

In [3]:
os.mkdir(path+'/temp')
os.chdir(path+'/temp')
path_drive_tmp = 'C:/Users/thiago.silva/TCC/RFB_CNPJ/temp'

In [4]:
#Informações de conexão com o banco Postgres
host='dataiesb.iesbtech.com.br'
dbname='IESB_Empresas'
user='1922120026_TCC_Thiago'
password='*********'

In [ ]:
# Criando uma barra de progresso para ser chamado no wget
def bar_progress(current, total, width=80):
    progress_message = "Downloading: %d%% [%d / %d] bytes" % (current / total * 100, current, total)
    sys.stdout.write("\r" + progress_message)
    sys.stdout.flush()


def etl_cnpj_rfb(links):
    for i in range(0, len(links)):
        filename = wget.download(links[i],path_drive_tmp,bar=bar_progress)

In [ ]:
etl_cnpj_rfb(link_estabelecimento)
#Deve ser repetido para todos os links obtidos

In [12]:
def unzip_cnpj_rfb(links):
    for i in range(0, len(links)):
        filename = [f for f in os.listdir(path_drive_tmp) if f.endswith('.zip')]
        filename = filename[0]
        print(filename)
        with ZipFile(filename, 'r') as zipObj:
            print('Extraindo arquivo zip...')
            zipObj.extractall(path=path_drive_tmp)

        tm.sleep(20)

        try:
            os.remove(filename)
        except OSError as e:
            print(e)
        else:
            pass

        tmp_file = os.listdir(path_drive_tmp)[0]
        filename_to_csv = tmp_file.split('.')[-1][:5] + str(i) + '.csv'

        os.rename(os.path.join(path_drive_tmp, tmp_file), os.path.join(path, filename_to_csv))

In [ ]:
unzip_cnpj_rfb(link_estabelecimento)
#Deve ser repetido para todos os links obtidos

In [6]:
# Função para criar conexão no banco
def conecta_db():
    con = psycopg2.connect(host='dataiesb.iesbtech.com.br', 
                         database='IESB_Empresas',
                         user='1922120026_TCC_Thiago', 
                         password='*********')
    return con


def postgres_test():
    try:
        con = conecta_db()
        con.close()
        print("Conexão com o banco de dados realizada com sucesso!")
    except:
        print("A conexão com o banco de dados falhou!")

postgres_test()

Conexão com o banco de dados realizada com sucesso!


In [5]:
# Função para criar tabela no banco
def criar_db(sql):
    con = conecta_db()
    cur = con.cursor()
    cur.execute(sql)
    con.commit()
    con.close()

In [17]:
# Criando a tabela para os dados de CNPJ ESTABELECIMENTO
sql = '''CREATE TABLE IF NOT EXISTS tb_estabelecimento (
cnpj_base varchar(255),
cnpj_ordem varchar(255),
cnpj_dv varchar(255),
cd_matriz_filial varchar(255),
nome_fantasia varchar(255),
cd_situacao_cadastral varchar(255),
dt_situacao_cadastral varchar(255),
cd_motivo_situacao_cadastral varchar(255),
cidade_exterior varchar(255),
cd_pais varchar(255),
dt_inicio_atividade varchar(255),
cd_cnae_principal varchar(255),
cd_cnae_secundario varchar(255),
tipo_logradouro varchar(255),
logradouro varchar(255),
numero varchar(255),
complemento varchar(255),
bairro varchar(255),
cep varchar(255),
uf varchar(255),
cd_municipio varchar(255),
ddd1 varchar(255),
telefone1 varchar(255),
ddd2 varchar(255),
telefone2 varchar(255),
ddd_fax varchar(255),
fax varchar(255),
email varchar(255),
situacao_especial varchar(255),
dt_situacao_especial varchar(255)
  );'''

criar_db(sql)

In [11]:
import chardet

def descobrir_encoding_arquivo_csv(caminho_arquivo):
    with open(caminho_arquivo, 'rb') as f:
        resultado = chardet.detect(f.read())
    return resultado['encoding']

# Exemplo de uso
caminho_arquivo = 'C:/Users/thiago.silva/TCC/RFB_CNPJ/ESTAB_1.csv'  # Substitua pelo caminho do seu arquivo CSV
encoding = descobrir_encoding_arquivo_csv(caminho_arquivo)
print("Encoding do arquivo:", encoding)

Encoding do arquivo: None


In [ ]:
engine = sqlalchemy.create_engine(f'postgresql://{user}:{password}@{host}/{dbname}')
engine = engine.connect()

#Tabela SIMPLES
colnames=['cnpj_base',
          'opcao_simples',
          'dt_opcao_simples',
          'dt_exclusao_simples',
          'opcao_mei',
          'dt_opcao_mei',
          'dt_exclusao_mei']

table_name = 'tb_simples'

# Fazendo leitura do CSV em partes
for chunk in pd.read_csv(path+'/SIMPLES.csv', delimiter=';', header=None, chunksize=65000, names=colnames, iterator=True, dtype=str, encoding="windows-1252"):

  # Formatando Colunas de Datas
  for i in chunk.columns[chunk.columns.str.contains('dt_')]:
    chunk.loc[chunk[i] == '00000000', i] = None
    chunk.loc[chunk[i] == '0', i] = None
    chunk[i] = pd.to_datetime(chunk[i], format='%Y%m%d', errors='coerce')

    # Usando Try para tentativas de conexão, caso perca a conexão espera 60 segundos para tentar novamente o envio
    try:
        chunk.to_sql(table_name,engine, if_exists="append", index=False)
    except:
        time.sleep(60)
        chunk.to_sql(table_name ,engine, if_exists="append", index=False)

engine.close()

print("### NOTE: FIM DO PROCESSO DE INSERÇÃO DOS DADOS NO BANCO!!!")

In [ ]:
engine = sqlalchemy.create_engine(f'postgresql://{user}:{password}@{host}/{dbname}')
engine = engine.connect()

#Tabela SOCIOS
colnames=['cnpj_base',
          'cd_tipo',
          'nome',
          'cpf_cnpj',
          'cd_qualificacao',
          'dt_entrada',
          'cd_pais',
          'representante',
          'nome_representante',
          'cd_qualificacao_representante',
          'cd_faixa_etaria']

### Inserindo dados no banco
table_name = 'tb_socio'

files = [f'SOCIO{i}.csv' for i in range(10)]

for i in files:
  # Fazendo leitura do CSV em partes
  for chunk in pd.read_csv(path+'/'+i, delimiter=';', header=None, chunksize=65000, names=colnames, iterator=True, dtype=str, encoding="windows-1252"):

    # Formatando Colunas de Datas
    for i in chunk.columns[chunk.columns.str.contains('dt_')]:
        chunk.loc[chunk[i] == '00000000', i] = None
        chunk.loc[chunk[i] == '0', i] = None
        chunk[i] = pd.to_datetime(chunk[i], format='%Y%m%d', errors='coerce')

        # Usando Try para tentativas de conexão, caso perca a conexão espera 60 segundos para tentar novamente o envio
        try:
            chunk.to_sql(table_name,engine, if_exists="append", index=False)
        except:
            time.sleep(60)
            chunk.to_sql(table_name ,engine, if_exists="append", index=False)

engine.close()

print("### NOTE: FIM DO PROCESSO DE INSERÇÃO DOS DADOS NO BANCO!!!")

In [ ]:
engine = sqlalchemy.create_engine(f'postgresql://{user}:{password}@{host}/{dbname}')
engine = engine.connect()

#Tabela ESTABELECIMENTOS
colnames=['cnpj_base',
          'cnpj_ordem',
          'cnpj_dv',
          'cd_matriz_filial',
          'nome_fantasia',
          'cd_situacao_cadastral',
          'dt_situacao_cadastral',
          'cd_motivo_situacao_cadastral',
          'cidade_exterior',
          'cd_pais',
          'dt_inicio_atividade',
          'cd_cnae_principal',
          'cd_cnae_secundario',
          'tipo_logradouro',
          'logradouro',
          'numero',
          'complemento',
          'bairro',
          'cep',
          'uf',
          'cd_municipio',
          'ddd1',
          'telefone1',
          'ddd2',
          'telefone2',
          'ddd_fax',
          'fax',
          'email',
          'situacao_especial',
          'dt_situacao_especial']

colunas_selecionadas = ['cidade_exterior','ddd1','telefone1','telefone2','fax','situacao_especial']

### Inserindo dados no banco
table_name = 'tb_estabelecimento'

files = [f'ESTAB_{i}.csv' for i in range(10)]

for i in files:
    # Fazendo leitura do CSV em partes
    for chunk in pd.read_csv(path+'/'+i, delimiter=';', header=None, chunksize=65000, names=colnames, iterator=True, dtype=str, encoding="ISO-8859-1"):
        print(path+'/'+i)
        # Formatação de colunas de datas
        columns_to_format = [col for col in chunk.columns if col.startswith('dt_')] + ['dt_situacao_especial']
        for col in columns_to_format:
            chunk.loc[chunk[col] == '00000000', col] = None
            chunk.loc[chunk[col] == '0', col] = None
            chunk[col] = pd.to_datetime(chunk[col], format='%Y%m%d', errors='coerce')
        
        # Converte as colunas selecionadas para o tipo de dados string
        chunk[colunas_selecionadas] = chunk[colunas_selecionadas].astype(str)

        # Remove as vírgulas nas colunas selecionadas
        chunk[colunas_selecionadas] = chunk[colunas_selecionadas].apply(lambda x: x.str.replace(',', ''))

        # Verificar se o valor é uma string e truncar se necessário
        chunk = chunk.applymap(lambda x: str(x)[:255] if isinstance(x, str) else x)
        chunk = chunk.replace('', np.nan)

        # Usando Try para tentativas de conexão, caso perca a conexão espera 60 segundos para tentar novamente o envio
        try:
            chunk.to_sql(table_name, engine, if_exists="append", index=False)
            print("Inserido no banco!")
        except:
            tm.sleep(60)
            chunk.to_sql(table_name, engine, if_exists="append", index=False)
            print("Inserido no banco!")

engine.close()

print("### NOTE: FIM DO PROCESSO DE INSERÇÃO DOS DADOS NO BANCO!!!")

In [ ]:
engine = sqlalchemy.create_engine(f'postgresql://{user}:{password}@{host}/{dbname}')
engine = engine.connect()

#Tabela EMPRESAS
colnames=['cnpj_base',
          'razao_social',
          'cd_natureza_juridica',
          'cd_qualificacao',
          'vl_capital_social',
          'cd_porte_empresa',
          'ente_federativo']

### Inserindo dados no banco
table_name = 'tb_empresa'

files = [f'EMPRE{i}.csv' for i in range(10)]

for i in files:
  # Fazendo leitura do CSV em partes
  for chunk in pd.read_csv(path+'/'+i, delimiter=';', header=None, chunksize=65000, names=colnames, iterator=True, dtype=str, encoding="windows-1252"):
    # Usando Try para tentativas de conexão, caso perca a conexão espera 60 segundos para tentar novamente o envio
    try:
      chunk.to_sql(table_name,engine, if_exists="append", index=False)
    except:
      time.sleep(60)
      chunk.to_sql(table_name ,engine, if_exists="append", index=False)

engine.close()

print("### NOTE: FIM DO PROCESSO DE INSERÇÃO DOS DADOS NO BANCO!!!")